In [183]:
import inspect
import operator as op

from functools import partial, wraps
from typing import *

In [215]:
def evaluable(dtype: Type[Any]) -> bool:
    return hasattr(dtype, "__bool__") or hasattr(dtype, "__len__")


def invertible(dtype: Type[Any]) -> bool:
    return hasattr(dtype, "__invert__")


class Curry:
    def __init__(self, func: Callable):
        self._func = func
    
    def __call__(self, *args, **kwargs):
        func = partial(self._func, *args, **kwargs)
        signature = inspect.signature(func)
        
        is_ready = True
        for param in signature.parameters.values():
            if param.kind in (param.POSITIONAL_ONLY, param.POSITIONAL_OR_KEYWORD) \
            and param.default == param.empty:
                is_ready = False
                
        if is_ready:
            return func()
            
        return Curry(func)
    
    def call(self, *args, **kwargs):
        return self._func(*args, **kwargs)
    
    @property
    def signature(self):
        return inspect.signature(self._func)
    
    @property
    def __signature__(self):
        return self.signature
    
    def __and__(self, other: "Curry"):
        assert evaluable(self.signature.return_annotation), "Only apply to return annotation type bool"
        assert evaluable(other.signature.return_annotation), "Only apply to return annotation type bool"
        
        def wrapper(*args, **kwargs) -> bool:
            if not self.call(*args, **kwargs):
                return False
            
            if not other.call(*args, **kwargs):
                return False
            
            return True
        
        return Curry(wrapper)
    
    def __or__(self, other: "Curry"):
        assert evaluable(self.signature.return_annotation), "Only apply to return annotation type bool"
        assert evaluable(other.signature.return_annotation), "Only apply to return annotation type bool"
        
        def wrapper(*args, **kwargs) -> bool:
            if self.call(*args, **kwargs):
                return True
            
            if other.call(*args, **kwargs):
                return True
            
            return False
        
        return Curry(wrapper)
    
    def __invert__(self):
        assert invertible(self.signature.return_annotation), "Return type is not invertible"
        
        def wrapper(*args, **kwargs) -> self.signature.return_annotation:
            return not self.call(*args, **kwargs)
        
        return Curry(wrapper)
    
    def __rshift__(self, other: "Curry"):
        def wrapper(*args, **kwargs):
            return other(self(*args, **kwargs))
        
        signature = self.signature.replace(return_annotation=other.signature.return_annotation)
        setattr(wrapper, "__signature__", signature)
        
        return Curry(wrapper)
    
    def __lshift__(self, other: "Curry"):
        return other >> self
    
    def __repr__(self):
        return self.signature.__repr__()

def curry(func: Callable):
    return Curry(func)


@curry
def eq(left: Any, right: Any) -> bool:
    return op.eq(left, right)


@curry
def gt(left: Any, right: Any) -> bool:
    return op.gt(left, right)


@curry
def lt(left: Any, right: Any) -> bool:
    return op.lt(left, right)


@curry
def ge(left: Any, right: Any) -> bool:
    return op.ge(left, right)


@curry
def le(left: Any, right: Any) -> bool:
    return op.le(left, right)


@curry
def is_instance(bases: Union[Type, Tuple[Type, ...]], obj: Any) -> bool:
    return isinstance(obj, bases)


@curry
def is_subclass(bases: Union[Type, Tuple[Type, ...]], cls: Type) -> bool:
    return issubclass(cls, bases)


@curry
def is_in(
    collection: Union[List[Any], Dict[Any, Any], Tuple[Any, ...]], val: Any
) -> bool:
    return val in collection


@curry
def peek_nth(index: int, seq: List[Any], default: Optional[Any] = Ellipsis) -> Any:
    if hasattr(seq, "__len__"):
        if 0 <= index < len(seq):
            return seq[index]
        return default

    for i, val in enumerate(seq):
        if i == index:
            return val
        
    if default == Ellipsis:
        raise IndexError("Index out of bound")

    return default


def all(*predicates: List[Callable[[Any], bool]]) -> Callable[[Any], bool]:
    def func(*args, **kwargs):
        for predicate in predicates:
            if not predicate(*args, **kwargs):
                return False
        return True

    return func


def any(*predicates: List[Callable[[Any], bool]]) -> Callable[[Any], bool]:
    def func(*args, **kwargs):
        for predicate in predicates:
            if predicate(*args, **kwargs):
                return True
        return False

    return func


@curry
def sort(iterable: Iterable[Any], key: Callable[[Any], Any]=None):
    return sorted(iterable, key=key)

In [210]:
gte = peek_nth(2) >> gt(right=10)

In [218]:
sort(key=lambda x: -x)([1, 1, 2])

[2, 1, 1]

In [161]:
a = 1

In [165]:
sig.replace(return_annotation=None)

<Signature (name: str, age: int, *args, **kwargs) -> None>

In [98]:
None.__bool__()

False

In [124]:
is_prime = lambda val: val == 1 or all(val % num for num in range(2, val))

In [129]:
is_prime(17)

True

In [148]:
print_pyramid = lambda val: print("\n".join(
    val[sum(range(i)):sum(range(i)) + i] 
    for i in range(1, len(val))
    if sum(range(i)) < len(val)
))

In [150]:
print_pyramid("abcdefghjk")

a
bc
def
ghjk
